Załadowanie biblioteki

In [64]:
library(tidyverse)

Załadowanie zbioru danych

In [67]:
dataset <- read.csv("mushroomdata/agaricus-lepiota.csv", header=FALSE, as.is=TRUE,
                               strip.white=TRUE, colClasses="character")

In [68]:
colnames(dataset) <- c("class", "cap_shape", "cap_surface", 
                        "cap_color", "bruises", "odor", 
                        "gill_attachement", "gill_spacing", "gill_size", 
                        "gill_color", "stalk_shape", "stalk_root", 
                        "stalk_surface_above_ring", "stalk_surface_below_ring", "stalk_color_above_ring", 
                        "stalk_color_below_ring", "veil_type", "veil_color", 
                        "ring_number", "ring_type", "spore_print_color", 
                        "population", "habitat")
dataset <- dataset %>% map_df(function(.x) as.factor(.x))
levels(dataset$class) <- c("edible", "poisonous")
levels(dataset$cap_shape) <- c("bell", "conical", "flat", "knobbed", "sunken", "convex")
levels(dataset$cap_color) <- c("buff", "cinnamon", "red", "gray", "brown", "pink", 
                                "green", "purple", "white", "yellow")
levels(dataset$cap_surface) <- c("fibrous", "grooves", "scaly", "smooth")
levels(dataset$bruises) <- c("no", "yes")
levels(dataset$odor) <- c("almond", "creosote", "foul", "anise", "musty", "none", "pungent", "spicy", "fishy")
levels(dataset$gill_attachement) <- c("attached", "free")
levels(dataset$gill_spacing) <- c("close", "crowded")
levels(dataset$gill_size) <- c("broad", "narrow")
levels(dataset$gill_color) <- c("buff", "red", "gray", "chocolate", "black", "brown", "orange", 
                                 "pink", "green", "purple", "white", "yellow")
levels(dataset$stalk_shape) <- c("enlarging", "tapering")
levels(dataset$stalk_root) <- c("missing", "bulbous", "club", "equal", "rooted")
levels(dataset$stalk_surface_above_ring) <- c("fibrous", "silky", "smooth", "scaly")
levels(dataset$stalk_surface_below_ring) <- c("fibrous", "silky", "smooth", "scaly")
levels(dataset$stalk_color_above_ring) <- c("buff", "cinnamon", "red", "gray", "brown", "pink", 
                                "green", "purple", "white", "yellow")
levels(dataset$stalk_color_below_ring) <- c("buff", "cinnamon", "red", "gray", "brown", "pink", 
                                "green", "purple", "white", "yellow")
levels(dataset$veil_type) <- "partial"
levels(dataset$veil_color) <- c("brown", "orange", "white", "yellow")
levels(dataset$ring_number) <- c("none", "one", "two")
levels(dataset$ring_type) <- c("evanescent", "flaring", "large", "none", "pendant")
levels(dataset$spore_print_color) <- c("buff", "chocolate", "black", "brown", "orange", 
                                        "green", "purple", "white", "yellow")
levels(dataset$population) <- c("abundant", "clustered", "numerous", "scattered", "several", "solitary")
levels(dataset$habitat) <- c("wood", "grasses", "leaves", "meadows", "paths", "urban", "waste")

dataset <- dataset %>% select(- veil_type)

[1] "e" "p"

In [54]:
set.seed(1810)
datasetsample <- caret::createDataPartition(y = dataset$class, times = 1, p = 0.1, list = FALSE)
train_dataset <- dataset[datasetsample, ]
test_dataset <- dataset[-datasetsample, ]

In [55]:
library(rpart)
set.seed(1810)
model_tree <- rpart(class ~ ., data = train_dataset, method = "class")
model_tree

n= 813 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 813 392 edible (0.51783518 0.48216482)  
  2) odor=almond,anise,none 427   6 edible (0.98594848 0.01405152) *
  3) odor=creosote,foul,musty,pungent,spicy,fishy 386   0 poisonous (0.00000000 1.00000000) *

In [26]:
caret::confusionMatrix(data=predict(model_tree, type = "class"), 
                       reference = train_dataset$class, 
                       positive="edible")

Confusion Matrix and Statistics

           Reference
Prediction  edible poisonous
  edible       421         6
  poisonous      0       386
                                         
               Accuracy : 0.9926         
                 95% CI : (0.984, 0.9973)
    No Information Rate : 0.5178         
    P-Value [Acc > NIR] : < 2e-16        
                                         
                  Kappa : 0.9852         
                                         
 Mcnemar's Test P-Value : 0.04123        
                                         
            Sensitivity : 1.0000         
            Specificity : 0.9847         
         Pos Pred Value : 0.9859         
         Neg Pred Value : 1.0000         
             Prevalence : 0.5178         
         Detection Rate : 0.5178         
   Detection Prevalence : 0.5252         
      Balanced Accuracy : 0.9923         
                                         
       'Positive' Class : edible         
                   

In [27]:
caret::confusionMatrix(data = predict(model_tree, newdata = test_dataset, type = "class"), 
                       reference = test_dataset$class, 
                       positive = "edible")

Confusion Matrix and Statistics

           Reference
Prediction  edible poisonous
  edible      3787       114
  poisonous      0      3410
                                          
               Accuracy : 0.9844          
                 95% CI : (0.9813, 0.9871)
    No Information Rate : 0.518           
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.9687          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 1.0000          
            Specificity : 0.9677          
         Pos Pred Value : 0.9708          
         Neg Pred Value : 1.0000          
             Prevalence : 0.5180          
         Detection Rate : 0.5180          
   Detection Prevalence : 0.5336          
      Balanced Accuracy : 0.9838          
                                          
       'Positive' Class : edible          

In [33]:
library(randomForest)
model_rf <- randomForest(class ~ ., ntree = 100, data = train_dataset)

In [34]:
print(model_rf)


Call:
 randomForest(formula = class ~ ., data = train_dataset, ntree = 100) 
               Type of random forest: classification
                     Number of trees: 100
No. of variables tried at each split: 4

        OOB estimate of  error rate: 0.12%
Confusion matrix:
          edible poisonous class.error
edible       421         0  0.00000000
poisonous      1       391  0.00255102


In [35]:
caret::confusionMatrix(data = model_rf$predicted, reference = train_dataset$class , 
                       positive = "edible")

Confusion Matrix and Statistics

           Reference
Prediction  edible poisonous
  edible       421         1
  poisonous      0       391
                                     
               Accuracy : 0.9988     
                 95% CI : (0.9932, 1)
    No Information Rate : 0.5178     
    P-Value [Acc > NIR] : <2e-16     
                                     
                  Kappa : 0.9975     
                                     
 Mcnemar's Test P-Value : 1          
                                     
            Sensitivity : 1.0000     
            Specificity : 0.9974     
         Pos Pred Value : 0.9976     
         Neg Pred Value : 1.0000     
             Prevalence : 0.5178     
         Detection Rate : 0.5178     
   Detection Prevalence : 0.5191     
      Balanced Accuracy : 0.9987     
                                     
       'Positive' Class : edible     
                                     

In [38]:
test_rf <- predict(model_rf, newdata = test_dataset)
table(test_rf, test_dataset$class)

           
test_rf     edible poisonous
  edible      3787        19
  poisonous      0      3505

In [43]:
library(e1071)
model_svm <- svm(class ~. , data=train_dataset, cost = 1000, gamma = 0.01)

In [44]:
test_svm <- predict(model_svm, newdata = test_dataset)

table(test_svm, test_dataset$class)

           
test_svm    edible poisonous
  edible      3787        26
  poisonous      0      3498

In [57]:
install.packages("naivebayes")

Warning message:
"pakiet 'naivebayes' jest w użyciu i nie zostanie zainstalowany"


In [58]:
library(naivebayes)
model_bayes <- naive_bayes(class ~ ., data = train_dataset, usekernel = T, laplace=1) 

In [47]:
p1 <- predict(model_bayes, train_mushroom)
table(p1, train_dataset$class)

ERROR: Error in predict.naive_bayes(model_bayes, train_mushroom): nie znaleziono obiektu 'train_mushroom'


In [48]:
p2 <- predict(model_bayes, test_dataset)
table(p2, test_dataset$class)

Warning message:
"predict.naive_bayes(): more features in the newdata are provided as there are probability tables in the object. Calculation is performed based on features to be found in the tables."


           
p2          edible poisonous
  edible      3726       405
  poisonous     61      3119

In [130]:
install.packages("MLmetrics")

Warning message:
"package 'MLmetrics' is in use and will not be installed"

In [131]:
library("MLmetrics")

ConfusionMatrix(p2, test_dataset$class)

ptest <- ifelse(p2 == "edible", 1, 0)
test_dataset_test <-ifelse(test_dataset$class == "edible", 1, 0)
AUC(ptest, test_dataset_test)
Accuracy(p2, test_dataset$class)
Precision(p2, test_dataset$class)
Recall(p2, test_dataset$class)
F1_Score(test_dataset$class, p2)
f2score <- FBeta_Score(test_dataset_test, ptest, positive = "1", beta = 2)
f2score

           y_pred
y_true      edible poisonous
  edible      3726        61
  poisonous    405      3119

[1] 0.934483

[1] 0.9362604

[1] 0.9838923

[1] 0.9019608

[1] 0.9411468

[1] 0.9663364